In [1]:
import pandas as pd
import json
import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ast
from tqdm import tqdm


# import docx
from docx import Document
from tabulate import tabulate
from IPython.display import display, HTML
import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('max_rows', 600)
pd.set_option('max_colwidth', 300)

In [2]:
%store -r rankings_H
# %store -r triples_arr

rankings = rankings_H
# triples = triples_arr
rankings

,statement,rank,score,probas_pos_neg
0,https://data.cooperationdatabank.org/vocab/pro...,1,7.381446,0.981578
1,https://data.cooperationdatabank.org/vocab/pro...,2,7.320162,0.980730
2,https://data.cooperationdatabank.org/vocab/pro...,2,7.317283,0.980689
3,https://data.cooperationdatabank.org/vocab/pro...,1,7.157279,0.978288
4,https://data.cooperationdatabank.org/vocab/pro...,1,7.092446,0.977233
...,...,...,...,...
555,https://data.cooperationdatabank.org/vocab/pro...,771,0.376581,0.220347
556,https://data.cooperationdatabank.org/vocab/pro...,1596,0.235451,0.202749
557,https://data.cooperationdatabank.org/vocab/pro...,1162,0.230638,0.202168
558,https://data.cooperationdatabank.org/vocab/pro...,3816,0.099160,0.186769


In [5]:
url = 'https://api.cooperationdatabank.org/datasets/coda-dev/databank/services/databank/sparql'

def get_sparql_dataframe(service, query):
    sparql = SPARQLWrapper(service)
    out = []
   
    sparql.setQuery(query)

    sparql.setReturnFormat(JSON)
    result = sparql.query()
    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
        
    return pd.DataFrame(out, columns=cols)

In [3]:
observations = pd.read_csv('data/observations_with_intervals_final.csv', index_col = 0)
observations = observations.reset_index(drop=True)

In [ ]:
data_table = pd.DataFrame(columns = ['obs'])
data_table = observations[['obs', 't1', 't2', 'independentProperties']]
data_table

In [ ]:
def construct_category_data_table(observations): 
    loop = tqdm(total = len(observations), position=0, leave=False)

    categories = pd.DataFrame()
    variables = pd.Series()
    for index, row in observations.iterrows(): 
        obs = row['obs']
        t1 = row['t1']
        t2 = row['t2']
        inds = row['independentProperties']
        print(inds)
        if len(inds) < 3: 
            
            for variable in inds:
                variables = variables.append(pd.Series(variable))
                prefix = """ PREFIX owl: <http://www.w3.org/2002/07/owl#>
                        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                        PREFIX class: <https://data.cooperationdatabank.org/vocab/class/>
                        PREFIX property: <https://data.cooperationdatabank.org/vocab/prop/>
                        PREFIX id: <https://data.cooperationdatabank.org/id/> 
                        PREFIX dct: <http://purl.org/dc/terms/>
                        """

                query = prefix + """
                             SELECT * WHERE {
                             """ + '<' + obs + '>' + """ property:treatment ?treatment . 
                             ?treatment """ + '<' + variable + '>' + """ ?categoryOrValue. 
                             """ + '<' + variable + '>' + """  rdfs:range ?range . 

                            }"""
                
                dfje = get_sparql_dataframe(url, query)
                dfje['IV'] = variable
                categories = categories.append(dfje).reset_index(drop=True)
            
        loop.set_description("Loading...".format(index))
        loop.update(1)
    
    loop.close()
    return categories, variables

categories, variables = construct_category_data_table(data_table)
categories

In [ ]:
pd.set_option('max_colwidth', 300)
categories

In [ ]:
categories.to_csv('data/categories.csv')

In [ ]:
variables.nunique()

In [ ]:
counts = categories.groupby(['IV', 'categoryOrValue']).size().reset_index(name='Count')
pd.set_option('max_rows', 30)
counts = counts.sort_values('IV')
counts

In [ ]:
%store -r categories
%store -r counts

In [ ]:
counts = pd.read_csv('data/counts.csv', index_col = 0)
categories = pd.read_csv('data/categories.csv', index_col = 0)

In [ ]:
categories

In [6]:
def create_category_output(catT1, catT2, var):
    categories = pd.Series([catT1, catT2])
    categories_placeholder = pd.Series()
    descriptions = pd.Series()
    range_type = query_range(var)
    treatment = pd.Series(['T1', 'T2'])
    
    if range_type != 'http://www.w3.org/2001/XMLSchema#string':
        for index, category in categories.items(): 
            try: 
                description = query_description(category)
            except: 
                continue
            if description != None: 
                slash = category.rfind('/')
                category_str = category[slash+1:]
                category_ser = pd.Series([category_str])
                category_ser.index = [index]
                categories_placeholder = categories_placeholder.append(category_ser)

                description = pd.Series([description])
                description.index = [index]
                descriptions = descriptions.append(description)

            else: 
                description = pd.Series([description])
                descriptions = descriptions.append(description)

    if descriptions.isna().any() == False and lenzi(descriptions) == False: 
        categories = categories_placeholder
        output = pd.DataFrame({'treatment' : treatment,
                               'IV value': categories, 
                               'description': descriptions}).reset_index(drop=True)
    else: 
        output = pd.DataFrame({'treatment' : treatment,
                               'IV value': categories}).reset_index(drop=True)
    return output

def disassemble_statement(statement): 
    result = statement.find(' ')
    item2_3 = statement[result+1:]
    result2 = item2_3.find(' ')

    item1 = statement[:result]
    item1a = item1[:item1.find('_H')]
    item2 = item2_3[:result2].strip()
    item3 = item2_3[result2:].strip()
    item2 = item2.replace('https://data.cooperationdatabank.org/vocab/prop/', '')
    return item1, item1a, item2, item3


def query_label(iri): 
    query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT * 
        WHERE {
         """ + '<' + iri + '>' + """ rdfs:label ?label . 
        }"""
    dfje = get_sparql_dataframe(url, query)
    try: 
        label = dfje['label'][0]
        return label
    except: 
        print("Except: No label found?")

def lenzi(df):
    return len(df.index) == 0

def query_description(entity): 
    prefix = """ PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX class: <https://data.cooperationdatabank.org/vocab/class/>
        PREFIX property: <https://data.cooperationdatabank.org/vocab/prop/>
        PREFIX id: <https://data.cooperationdatabank.org/id/> 
        PREFIX dct: <http://purl.org/dc/terms/>
"""
    query = prefix + """
            SELECT * 
            WHERE {
             """ + '<' + entity + '>' + """ dct:description ?description . 
            }"""
    
    dfje = get_sparql_dataframe(url, query)
   
    if lenzi(dfje) == False: 
        description = dfje['description']
        return description.item()



def effect_string_maker(effect):
    if effect == 'hasPositiveEffectOn': 
        effect_string = 'has positive effect on'
        return effect_string
    elif effect == 'hasNegativeEffectOn': 
        effect_string = 'has negative effect on'
        return effect_string

def query_range(IV): 
    query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT * 
        WHERE {
         """ + '<' + IV + '>' + """ rdfs:range ?range . 
        }"""
    dfje = get_sparql_dataframe(url, query)

    if dfje.empty == False:
        IVrange = dfje['range'][0]
        return IVrange

    

    
def control_function( rankings, obs):        
    for index, row in rankings.iterrows(): 
        IV_H, IV, effect, DV = disassemble_statement(row['statement'])
        effect_string = effect_string_maker(effect)
        
        dois = obs[obs['IV_new'].str.contains(str(IV_H))]['doi'].unique().tolist()
        catT1 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT1'].unique().tolist()[0]
        catT2 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT2'].unique().tolist()[0]
        
        output = create_category_output(catT1, catT2, IV)
        print('\n\nHypothesis ' + str(index+1) + ':', query_label(IV), effect_string, query_label(DV).lower(),
              '\n\nDependent Variable (DV): ', DV, 
              '\n\nIndependent Variable (IV): ', IV, IV_H, 
              '\nDescription IV: ', query_description(IV), 
              '\n\nThe IV categories that were researched and their descriptions are shown below: ')

        with pd.option_context('display.width', 10):  # more options can be specified also
            display(output.style.hide_index())
            print(output.to_latex())
    
        print('\nThe paper(s) that researched this topic is/are the following: ')
        for doi in dois: 
            if doi == 'nan':
                print('Papers are not available in the data') 
            else: 
                print(doi)

# Most likely examples 
print('POSSIBLE NEW HYPOTHESES WITH HIGHEST PROBABILITIES')
control_function(rankings[:10], observations)
# least likely examples 
print('\n\nHYPOTHESES WITH VERY LOW PROBABILITIES')
control_function(rankings[len(rankings)-10:], observations)

POSSIBLE NEW HYPOTHESES WITH HIGHEST PROBABILITIES


c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.




Hypothesis 1: iterated strategy has positive effect on cooperation 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/cooperation 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/iteratedStrategy https://data.cooperationdatabank.org/vocab/prop/iteratedStrategy_H6 
Description IV:  The specific strategy played by the participant's partner in an iterated game. 

The IV categories that were researched and their descriptions are shown below: 


treatment,IV value,description
T1,preprogrammed_cooperation_rate,The partner cooperates randomly with a given cooperation rate across the duration of the game
T2,other,Other strategies


\begin{tabular}{llll}
\toprule
{} & treatment &                        IV value &                                        description \\
\midrule
0 &        T1 &  preprogrammed\_cooperation\_rate &  The partner cooperates randomly with a given c... \\
1 &        T2 &                           other &                                   Other strategies \\
\bottomrule
\end{tabular}


The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1080/00224545.1970.9922453
http://dx.doi.org/10.1177/0022002783027003007
http://dx.doi.org/10.1177/0022002794038004006
http://dx.doi.org/10.1177/002200276901300108


...

\begin{tabular}{llll}
\toprule
{} & treatment &       IV value &                                        description \\
\midrule
0 &        T1 &    tit-for-tat &  The partner cooperates on the first trial, the... \\
1 &        T2 &  tit-for-tat+1 &  The partner cooperates on the first trial, and... \\
\bottomrule
\end{tabular}


The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1037/0022-3514.87.6.845
http://dx.doi.org/10.1037//0022-3514.82.5.768


Hypothesis 559: uncertainty target has positive effect on cooperation 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/cooperation 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/uncertaintyTarget https://data.cooperationdatabank.org/vocab/prop/uncertaintyTarget_H1 
Description IV:  Whether participants receive uncertain information about a certain aspect of the game. This also includes receiving detailed vs less specific information, as well 

treatment,IV value,description
T1,loss,"Negative payoff that resulted in a loss of resources due to endogenous (e.g., not reaching a threshold) or exogenous (e.g., shocks) factors"
T2,threshold,"The minimum threshold of total contribution by all group members for the public good to be provided or the occurrence of a loss (e.g., in case of public bad games)"


\begin{tabular}{llll}
\toprule
{} & treatment &   IV value &                                        description \\
\midrule
0 &        T1 &       loss &  Negative payoff that resulted in a loss of res... \\
1 &        T2 &  threshold &  The minimum threshold of total contribution by... \\
\bottomrule
\end{tabular}


The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1073/pnas.1208417109


Hypothesis 560: exit option has negative effect on contributions 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/contributions 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/exitOption https://data.cooperationdatabank.org/vocab/prop/exitOption_H1 
Description IV:  Whether the game provided the option to walk away and not interact in the game. TRUE = Exit option is provided, FALSE = Exit option is not provided 

The IV categories that were researched and their descriptions are shown below: 


treatment,IV value
T1,0
T2,1


\begin{tabular}{lll}
\toprule
{} & treatment & IV value \\
\midrule
0 &        T1 &        0 \\
1 &        T2 &        1 \\
\bottomrule
\end{tabular}


The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1371/journal.pone.0069871
http://dx.doi.org/10.1037/h0024598


In [ ]:
observations

In [ ]:
from docx.enum.dml import MSO_THEME_COLOR_INDEX
import docx
def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run ()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = True

    return hyperlink

In [ ]:
def control_function2(document, hnumber, rankings, obs):        
    for index, row in rankings.iterrows(): 
        IV_H, IV, effect, DV = disassemble_statement(row['statement'])
        effect_string = effect_string_maker(effect)

        dois = obs[obs['IV_new'].str.contains(str(IV_H))]['doi'].unique().tolist()
        catT1 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT1'].unique().tolist()[0]
        catT2 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT2'].unique().tolist()[0]
        
        output = create_category_output(catT1, catT2, IV)
        document.add_heading(('Hypothesis ' + str(hnumber) + ': ' + query_label(IV) + ' ' + effect_string + ' ' + query_label(DV).lower()), level=1)
        p = document.add_paragraph('Dependent Variable (DV): ')
        add_hyperlink(p, str(DV), DV)

        p = document.add_paragraph('\nInependent Variable (IV): ')
        add_hyperlink(p, str(IV), IV)
            
        document.add_paragraph('Description IV: ' + query_description(IV) + 
                               '\n\nThe IV values that were researched and (if available) their descriptions are shown below: ')
        
        t = document.add_table(output.shape[0]+1, output.shape[1])
        
        # add the header rows.
        for j in range(output.shape[-1]):
            t.cell(0,j).text = output.columns[j]

        # add the rest of the data frame
        for i in range(output.shape[0]):
            for j in range(output.shape[-1]):
                t.cell(i+1,j).text = str(output.values[i,j])
                
        document.add_paragraph('\nThe paper(s) that researched this topic is/are the following: ')
        for doi in dois: 
            if type(doi) == float:
                continue
            else:
                p = document.add_paragraph('')
                add_hyperlink(p, str(doi), doi)
                
    document.save('hypotheses.docx')


# # Most likely examples 
# print('POSSIBLE NEW HYPOTHESES WITH HIGHEST PROBABILITIES')
# control_function(counts, rankings[:10], observations)
# # least likely examples 
# print('\n\nHYPOTHESES WITH VERY LOW PROBABILITIES')
# control_function(counts, rankings[len(rankings)-10:], observations)

In [ ]:
import random

indexes = [0,1,2,3,4,5,6,7,8,9,550,551,552,553,554,555,556,557,558,559]

def random_index(data):
    choice = random.choice(data) 
    data.remove(choice) 
    return choice, data 

# choice, indexjes = simple(indexes)

document = Document()
document.add_heading('Hypotheses', 0)
hnumber = 0
for r in range(len(indexes)): 
    hnumber += 1
    choice, indexes = random_index(indexes)
    control_function2(document, hnumber, rankings[choice:choice+1], observations)
    print(hnumber, choice)